# 🎭 Detecção de Máscara Facial com CNN para ESP32-CAM

## Projeto de PDI (Processamento Digital de Imagens)

Este notebook implementa um sistema de detecção de máscara facial usando **Redes Neurais Convolucionais (CNN)**.

### O que você vai aprender:
1. Como carregar e visualizar um dataset de imagens
2. Como pré-processar imagens para Machine Learning
3. Como criar uma CNN leve e otimizada
4. Como treinar e avaliar o modelo
5. Como converter para TensorFlow Lite (para embarcar na ESP32)

---

## 📦 1. Instalação das Bibliotecas

Primeiro, vamos instalar todas as bibliotecas necessárias. Execute esta célula apenas uma vez.

In [ ]:
# Instala as bibliotecas necessárias (execute apenas uma vez)
!pip install tensorflow numpy matplotlib scikit-learn opencv-python pillow seaborn

## 📚 2. Importação das Bibliotecas

Aqui importamos todas as bibliotecas que serão usadas no projeto:

- **TensorFlow/Keras**: Framework para criar e treinar a CNN
- **NumPy**: Manipulação de arrays numéricos
- **Matplotlib/Seaborn**: Visualização de dados e gráficos
- **OpenCV (cv2)**: Processamento de imagens
- **Scikit-learn**: Divisão de dados e métricas

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image

# TensorFlow e Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Scikit-learn para divisão de dados e métricas
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Configurações para visualização
plt.style.use('ggplot')
%matplotlib inline

# Verificar se TensorFlow está usando GPU (se disponível)
print(f"TensorFlow versão: {tf.__version__}")
print(f"GPU disponível: {tf.config.list_physical_devices('GPU')}")

## 🗂️ 3. Configuração dos Caminhos

Aqui definimos os caminhos para o dataset e os parâmetros importantes do projeto.

### Parâmetros Escolhidos:
- **IMG_SIZE = 96x96**: Tamanho ideal para ESP32-CAM (balanceia qualidade vs performance)
- **Grayscale (1 canal)**: Reduz tamanho do modelo em 3x comparado a RGB
- **Batch size = 16**: Pequeno para caber na memória limitada

In [ ]:
# ============================================
# CONFIGURAÇÕES DO PROJETO
# ============================================

# Caminho para o dataset
DATASET_PATH = 'dataset_mascaras/data'

# Parâmetros das imagens
IMG_SIZE = 96          # Tamanho da imagem (96x96 pixels)
IMG_CHANNELS = 1       # 1 = Grayscale, 3 = RGB

# Parâmetros de treinamento
BATCH_SIZE = 16        # Quantidade de imagens por lote
EPOCHS = 30            # Número de épocas de treinamento
VALIDATION_SPLIT = 0.2 # 20% dos dados para validação
TEST_SPLIT = 0.1       # 10% dos dados para teste

# Classes do problema
CLASSES = ['sem_mascara', 'com_mascara']
NUM_CLASSES = len(CLASSES)

# Seed para reprodutibilidade
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print("✅ Configurações carregadas!")
print(f"   📁 Dataset: {DATASET_PATH}")
print(f"   📐 Tamanho das imagens: {IMG_SIZE}x{IMG_SIZE}")
print(f"   🎨 Canais: {'Grayscale' if IMG_CHANNELS == 1 else 'RGB'}")
print(f"   📦 Batch size: {BATCH_SIZE}")
print(f"   🔄 Épocas: {EPOCHS}")

## 📊 4. Exploração do Dataset

Antes de treinar o modelo, é importante entender nosso dataset:
- Quantas imagens temos de cada classe?
- Como são as imagens?
- O dataset está balanceado?

In [ ]:
# Conta quantas imagens temos em cada pasta
with_mask_path = os.path.join(DATASET_PATH, 'with_mask')
without_mask_path = os.path.join(DATASET_PATH, 'without_mask')

# Lista os arquivos de cada classe
with_mask_files = [f for f in os.listdir(with_mask_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
without_mask_files = [f for f in os.listdir(without_mask_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

n_with_mask = len(with_mask_files)
n_without_mask = len(without_mask_files)
total_images = n_with_mask + n_without_mask

print("="*50)
print("📊 ESTATÍSTICAS DO DATASET")
print("="*50)
print(f"\n✅ Com máscara (with_mask):    {n_with_mask} imagens")
print(f"❌ Sem máscara (without_mask): {n_without_mask} imagens")
print(f"\n📸 Total de imagens: {total_images}")
print(f"\n⚖️  Proporção: {n_with_mask/total_images*100:.1f}% com máscara / {n_without_mask/total_images*100:.1f}% sem máscara")

In [ ]:
# Visualização da distribuição das classes
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Gráfico de barras
classes_labels = ['Com Máscara', 'Sem Máscara']
counts = [n_with_mask, n_without_mask]
colors = ['#2ecc71', '#e74c3c']

axes[0].bar(classes_labels, counts, color=colors, edgecolor='black')
axes[0].set_title('Distribuição das Classes', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Quantidade de Imagens')
for i, (count, label) in enumerate(zip(counts, classes_labels)):
    axes[0].text(i, count + 5, str(count), ha='center', fontweight='bold')

# Gráfico de pizza
axes[1].pie(counts, labels=classes_labels, colors=colors, autopct='%1.1f%%',
            startangle=90, explode=(0.05, 0.05))
axes[1].set_title('Proporção das Classes', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('distribuicao_dataset.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💾 Gráfico salvo como 'distribuicao_dataset.png'")

## 🖼️ 5. Visualização de Amostras

Vamos ver algumas imagens do dataset para entender como são os dados.

In [ ]:
def mostrar_amostras(pasta, titulo, n_amostras=5):
    """
    Mostra amostras de imagens de uma pasta.
    
    Args:
        pasta: Caminho para a pasta com as imagens
        titulo: Título para o gráfico
        n_amostras: Número de amostras a mostrar
    """
    arquivos = [f for f in os.listdir(pasta) if f.endswith(('.jpg', '.jpeg', '.png'))][:n_amostras]
    
    fig, axes = plt.subplots(1, n_amostras, figsize=(15, 3))
    fig.suptitle(titulo, fontsize=14, fontweight='bold')
    
    for idx, arquivo in enumerate(arquivos):
        img_path = os.path.join(pasta, arquivo)
        img = Image.open(img_path)
        
        axes[idx].imshow(img)
        axes[idx].axis('off')
        axes[idx].set_title(f'{img.size[0]}x{img.size[1]}', fontsize=10)
    
    plt.tight_layout()
    plt.show()

# Mostra amostras de cada classe
print("\n🎭 AMOSTRAS COM MÁSCARA:\n")
mostrar_amostras(with_mask_path, '✅ Imagens COM Máscara')

print("\n😷 AMOSTRAS SEM MÁSCARA:\n")
mostrar_amostras(without_mask_path, '❌ Imagens SEM Máscara')

## 🔧 6. Pré-processamento das Imagens

Agora vamos preparar as imagens para o treinamento:

1. **Redimensionar** para 96x96 pixels (padrão para ESP32-CAM)
2. **Converter para grayscale** (economia de memória)
3. **Normalizar** valores de 0-255 para 0-1
4. **Criar labels** (0 = sem máscara, 1 = com máscara)

### Por que 96x96?
- A ESP32-CAM tem apenas ~520KB de RAM
- Imagens menores = modelo menor = mais rápido
- 96x96 é um bom equilíbrio entre qualidade e performance

In [ ]:
def carregar_e_preprocessar_imagem(caminho_imagem, tamanho=IMG_SIZE, grayscale=True):
    """
    Carrega uma imagem e aplica o pré-processamento.
    
    Args:
        caminho_imagem: Caminho para o arquivo da imagem
        tamanho: Tamanho final da imagem (tamanho x tamanho)
        grayscale: Se True, converte para escala de cinza
    
    Returns:
        Imagem pré-processada como array numpy
    """
    # Lê a imagem
    if grayscale:
        img = cv2.imread(caminho_imagem, cv2.IMREAD_GRAYSCALE)
    else:
        img = cv2.imread(caminho_imagem)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Redimensiona para o tamanho desejado
    img = cv2.resize(img, (tamanho, tamanho))
    
    # Normaliza para valores entre 0 e 1
    img = img.astype('float32') / 255.0
    
    return img

print("✅ Função de pré-processamento definida!")

In [ ]:
def carregar_dataset(dataset_path, img_size=IMG_SIZE, grayscale=True):
    """
    Carrega todo o dataset e aplica pré-processamento.
    
    Args:
        dataset_path: Caminho para a pasta do dataset
        img_size: Tamanho das imagens
        grayscale: Se True, converte para grayscale
    
    Returns:
        X: Array com as imagens
        y: Array com as labels (0 = sem máscara, 1 = com máscara)
    """
    imagens = []
    labels = []
    
    # Carrega imagens SEM máscara (label = 0)
    print("📂 Carregando imagens SEM máscara...")
    pasta_sem_mascara = os.path.join(dataset_path, 'without_mask')
    for arquivo in os.listdir(pasta_sem_mascara):
        if arquivo.endswith(('.jpg', '.jpeg', '.png')):
            caminho = os.path.join(pasta_sem_mascara, arquivo)
            try:
                img = carregar_e_preprocessar_imagem(caminho, img_size, grayscale)
                imagens.append(img)
                labels.append(0)  # 0 = sem máscara
            except Exception as e:
                print(f"   ⚠️ Erro ao carregar {arquivo}: {e}")
    
    # Carrega imagens COM máscara (label = 1)
    print("📂 Carregando imagens COM máscara...")
    pasta_com_mascara = os.path.join(dataset_path, 'with_mask')
    for arquivo in os.listdir(pasta_com_mascara):
        if arquivo.endswith(('.jpg', '.jpeg', '.png')):
            caminho = os.path.join(pasta_com_mascara, arquivo)
            try:
                img = carregar_e_preprocessar_imagem(caminho, img_size, grayscale)
                imagens.append(img)
                labels.append(1)  # 1 = com máscara
            except Exception as e:
                print(f"   ⚠️ Erro ao carregar {arquivo}: {e}")
    
    # Converte para arrays numpy
    X = np.array(imagens)
    y = np.array(labels)
    
    # Adiciona dimensão do canal se grayscale
    if grayscale and len(X.shape) == 3:
        X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
    
    print(f"\n✅ Dataset carregado com sucesso!")
    print(f"   📊 Shape das imagens: {X.shape}")
    print(f"   📊 Shape das labels: {y.shape}")
    
    return X, y

# Carrega o dataset
X, y = carregar_dataset(DATASET_PATH, IMG_SIZE, grayscale=(IMG_CHANNELS == 1))

In [ ]:
# Visualiza algumas imagens pré-processadas
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Imagens Pré-processadas (96x96 Grayscale)', fontsize=14, fontweight='bold')

# Índices aleatórios para visualização
indices_sem_mascara = np.where(y == 0)[0][:5]
indices_com_mascara = np.where(y == 1)[0][:5]

# Mostra imagens sem máscara (primeira linha)
for i, idx in enumerate(indices_sem_mascara):
    axes[0, i].imshow(X[idx].squeeze(), cmap='gray')
    axes[0, i].axis('off')
    axes[0, i].set_title('Sem Máscara', color='red', fontsize=10)

# Mostra imagens com máscara (segunda linha)
for i, idx in enumerate(indices_com_mascara):
    axes[1, i].imshow(X[idx].squeeze(), cmap='gray')
    axes[1, i].axis('off')
    axes[1, i].set_title('Com Máscara', color='green', fontsize=10)

plt.tight_layout()
plt.savefig('imagens_preprocessadas.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💾 Gráfico salvo como 'imagens_preprocessadas.png'")

## ✂️ 7. Divisão dos Dados

Dividimos o dataset em três partes:

- **Treino (70%)**: Usado para treinar a rede neural
- **Validação (20%)**: Usado para ajustar hiperparâmetros e evitar overfitting
- **Teste (10%)**: Usado apenas no final para avaliar o modelo

⚠️ **Importante**: O conjunto de teste NUNCA é visto durante o treinamento!

In [ ]:
# Primeiro, separa conjunto de teste
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, 
    test_size=TEST_SPLIT, 
    random_state=RANDOM_SEED,
    stratify=y  # Mantém a proporção das classes
)

# Depois, separa treino e validação
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=VALIDATION_SPLIT / (1 - TEST_SPLIT),  # Ajusta a proporção
    random_state=RANDOM_SEED,
    stratify=y_temp
)

print("="*50)
print("📊 DIVISÃO DO DATASET")
print("="*50)
print(f"\n🏋️ Treino:     {len(X_train)} imagens ({len(X_train)/len(X)*100:.1f}%)")
print(f"   - Sem máscara: {np.sum(y_train == 0)}")
print(f"   - Com máscara: {np.sum(y_train == 1)}")

print(f"\n📋 Validação:  {len(X_val)} imagens ({len(X_val)/len(X)*100:.1f}%)")
print(f"   - Sem máscara: {np.sum(y_val == 0)}")
print(f"   - Com máscara: {np.sum(y_val == 1)}")

print(f"\n🧪 Teste:      {len(X_test)} imagens ({len(X_test)/len(X)*100:.1f}%)")
print(f"   - Sem máscara: {np.sum(y_test == 0)}")
print(f"   - Com máscara: {np.sum(y_test == 1)}")

## 🧠 8. Criação da CNN (Rede Neural Convolucional)

Agora vamos criar a arquitetura da CNN. É uma rede **leve e otimizada** para rodar na ESP32-CAM.

### Arquitetura da Rede:

```
Entrada: 96x96x1 (imagem grayscale)
    ↓
Conv2D (8 filtros, 3x3) → BatchNorm → ReLU → MaxPool
    ↓
Conv2D (16 filtros, 3x3) → BatchNorm → ReLU → MaxPool
    ↓
Conv2D (32 filtros, 3x3) → BatchNorm → ReLU → MaxPool
    ↓
Flatten → Dense (32) → Dropout (0.5)
    ↓
Dense (1) → Sigmoid
    ↓
Saída: 0 (sem máscara) ou 1 (com máscara)
```

### Por que esta arquitetura?
- **Poucos filtros**: Reduz o tamanho do modelo
- **BatchNormalization**: Acelera o treinamento e estabiliza
- **Dropout**: Previne overfitting com dataset pequeno
- **Saída sigmoid**: Classificação binária

In [ ]:
def criar_modelo_cnn(input_shape=(IMG_SIZE, IMG_SIZE, IMG_CHANNELS)):
    """
    Cria uma CNN leve otimizada para ESP32-CAM.
    
    Args:
        input_shape: Formato da entrada (altura, largura, canais)
    
    Returns:
        Modelo Keras compilado
    """
    model = models.Sequential([
        # Camada de entrada
        layers.Input(shape=input_shape),
        
        # Bloco Convolucional 1
        layers.Conv2D(8, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Bloco Convolucional 2
        layers.Conv2D(16, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Bloco Convolucional 3
        layers.Conv2D(32, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Camadas Densas (Fully Connected)
        layers.Flatten(),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.5),  # Previne overfitting
        
        # Camada de saída (classificação binária)
        layers.Dense(1, activation='sigmoid')
    ])
    
    # Compila o modelo
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Cria o modelo
modelo = criar_modelo_cnn()

# Mostra o resumo da arquitetura
print("\n" + "="*60)
print("🧠 ARQUITETURA DA CNN")
print("="*60)
modelo.summary()

In [ ]:
# Calcula o tamanho do modelo
num_params = modelo.count_params()
tamanho_estimado_mb = (num_params * 4) / (1024 * 1024)  # 4 bytes por parâmetro (float32)

print("\n" + "="*50)
print("📏 TAMANHO DO MODELO")
print("="*50)
print(f"\n🔢 Total de parâmetros: {num_params:,}")
print(f"📦 Tamanho estimado: {tamanho_estimado_mb:.2f} MB (float32)")
print(f"📦 Tamanho quantizado: ~{tamanho_estimado_mb/4:.2f} MB (int8)")

if tamanho_estimado_mb < 1:
    print("\n✅ Modelo adequado para ESP32-CAM!")
else:
    print("\n⚠️ Modelo pode ser grande demais. Considere reduzir os filtros.")

## 🔄 9. Data Augmentation (Aumento de Dados)

Como nosso dataset é pequeno, usamos **data augmentation** para criar variações das imagens:

- **Rotação**: Pequenas rotações (-10° a +10°)
- **Zoom**: Pequeno zoom (90% a 110%)
- **Flip horizontal**: Espelha a imagem
- **Shift**: Pequenos deslocamentos

Isso ajuda o modelo a generalizar melhor e **previne overfitting**.

In [ ]:
# Cria geradores de data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=10,           # Rotação de -10° a +10°
    width_shift_range=0.1,       # Deslocamento horizontal
    height_shift_range=0.1,      # Deslocamento vertical
    zoom_range=0.1,              # Zoom de 90% a 110%
    horizontal_flip=True,        # Espelhamento horizontal
    fill_mode='nearest'          # Preenche pixels vazios
)

# Para validação, não aplicamos augmentation
val_datagen = ImageDataGenerator()

print("✅ Data augmentation configurado!")
print("\n📷 Transformações aplicadas no treino:")
print("   • Rotação: ±10°")
print("   • Shift: ±10%")
print("   • Zoom: ±10%")
print("   • Flip horizontal: Sim")

In [ ]:
# Visualiza exemplos de data augmentation
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Exemplos de Data Augmentation', fontsize=14, fontweight='bold')

# Pega uma imagem de exemplo
img_exemplo = X_train[0:1]

# Gera 10 variações
augmented_images = [train_datagen.random_transform(img_exemplo[0]) for _ in range(10)]

for i, img in enumerate(augmented_images):
    row = i // 5
    col = i % 5
    axes[row, col].imshow(img.squeeze(), cmap='gray')
    axes[row, col].axis('off')
    axes[row, col].set_title(f'Variação {i+1}', fontsize=10)

plt.tight_layout()
plt.savefig('data_augmentation.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💾 Gráfico salvo como 'data_augmentation.png'")

## 🏋️ 10. Treinamento do Modelo

Agora vamos treinar a CNN! Usamos duas técnicas importantes:

1. **Early Stopping**: Para o treinamento se a validação não melhorar
2. **Model Checkpoint**: Salva o melhor modelo durante o treinamento

In [ ]:
# Callbacks para monitorar o treinamento
callbacks = [
    # Para o treino se a validação não melhorar por 5 épocas
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    # Salva o melhor modelo
    ModelCheckpoint(
        'melhor_modelo.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("✅ Callbacks configurados!")
print("   • EarlyStopping: patience=5")
print("   • ModelCheckpoint: salva em 'melhor_modelo.keras'")

In [ ]:
# Treina o modelo
print("\n" + "="*60)
print("🏋️ INICIANDO TREINAMENTO")
print("="*60)
print(f"\n⏱️ Épocas máximas: {EPOCHS}")
print(f"📦 Batch size: {BATCH_SIZE}")
print(f"🔄 Data Augmentation: Ativado\n")

# Cria os geradores
train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)
val_generator = val_datagen.flow(X_val, y_val, batch_size=BATCH_SIZE)

# Treina!
historico = modelo.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*60)
print("✅ TREINAMENTO CONCLUÍDO!")
print("="*60)

## 📈 11. Visualização do Treinamento

Vamos ver como o modelo aprendeu ao longo das épocas.

In [ ]:
def plotar_historico(historico):
    """
    Plota os gráficos de loss e accuracy durante o treinamento.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Gráfico de Loss
    axes[0].plot(historico.history['loss'], label='Treino', linewidth=2)
    axes[0].plot(historico.history['val_loss'], label='Validação', linewidth=2)
    axes[0].set_title('Loss ao Longo do Treinamento', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Época')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Gráfico de Accuracy
    axes[1].plot(historico.history['accuracy'], label='Treino', linewidth=2)
    axes[1].plot(historico.history['val_accuracy'], label='Validação', linewidth=2)
    axes[1].set_title('Acurácia ao Longo do Treinamento', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Época')
    axes[1].set_ylabel('Acurácia')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('historico_treinamento.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n💾 Gráfico salvo como 'historico_treinamento.png'")

# Plota o histórico
plotar_historico(historico)

## 🧪 12. Avaliação do Modelo

Agora vamos avaliar o modelo no conjunto de **teste** (dados que ele nunca viu!).

In [ ]:
# Avalia no conjunto de teste
print("\n" + "="*60)
print("🧪 AVALIAÇÃO NO CONJUNTO DE TESTE")
print("="*60)

test_loss, test_accuracy = modelo.evaluate(X_test, y_test, verbose=0)

print(f"\n📊 Resultados no Conjunto de Teste:")
print(f"   • Loss: {test_loss:.4f}")
print(f"   • Acurácia: {test_accuracy*100:.2f}%")

if test_accuracy >= 0.90:
    print("\n🎉 Excelente! Acurácia acima de 90%!")
elif test_accuracy >= 0.80:
    print("\n✅ Bom! Acurácia acima de 80%!")
else:
    print("\n⚠️ Acurácia abaixo de 80%. Considere mais dados ou ajustes.")

In [ ]:
# Faz predições no conjunto de teste
y_pred_prob = modelo.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Sem Máscara', 'Com Máscara'],
            yticklabels=['Sem Máscara', 'Com Máscara'])
plt.title('Matriz de Confusão', fontsize=14, fontweight='bold')
plt.xlabel('Predição')
plt.ylabel('Real')
plt.tight_layout()
plt.savefig('matriz_confusao.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💾 Gráfico salvo como 'matriz_confusao.png'")

In [ ]:
# Relatório de Classificação
print("\n" + "="*60)
print("📋 RELATÓRIO DE CLASSIFICAÇÃO")
print("="*60)
print(classification_report(y_test, y_pred, target_names=['Sem Máscara', 'Com Máscara']))

## 🔮 13. Testando com Novas Imagens

Vamos criar uma função para testar o modelo com imagens individuais.

In [ ]:
def prever_mascara(caminho_imagem, modelo, mostrar=True):
    """
    Faz a predição para uma única imagem.
    
    Args:
        caminho_imagem: Caminho para a imagem
        modelo: Modelo treinado
        mostrar: Se True, mostra a imagem com a predição
    
    Returns:
        Predição (0 = sem máscara, 1 = com máscara) e probabilidade
    """
    # Carrega e pré-processa a imagem
    img = carregar_e_preprocessar_imagem(caminho_imagem, IMG_SIZE, grayscale=True)
    img_batch = img.reshape(1, IMG_SIZE, IMG_SIZE, 1)
    
    # Faz a predição
    probabilidade = modelo.predict(img_batch, verbose=0)[0][0]
    predicao = 1 if probabilidade > 0.5 else 0
    
    # Mostra a imagem se solicitado
    if mostrar:
        plt.figure(figsize=(6, 6))
        
        # Carrega a imagem original para visualização
        img_original = Image.open(caminho_imagem)
        plt.imshow(img_original)
        
        if predicao == 1:
            titulo = f"✅ COM MÁSCARA ({probabilidade*100:.1f}%)"
            cor = 'green'
        else:
            titulo = f"❌ SEM MÁSCARA ({(1-probabilidade)*100:.1f}%)"
            cor = 'red'
        
        plt.title(titulo, fontsize=14, fontweight='bold', color=cor)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
    
    return predicao, probabilidade

print("✅ Função de predição criada!")

In [ ]:
# Testa com algumas imagens do dataset
print("\n" + "="*60)
print("🔮 TESTANDO PREDIÇÕES")
print("="*60)

# Testa imagem com máscara
img_com_mascara = os.path.join(with_mask_path, with_mask_files[0])
print(f"\n📸 Testando: {with_mask_files[0]}")
prever_mascara(img_com_mascara, modelo)

# Testa imagem sem máscara
img_sem_mascara = os.path.join(without_mask_path, without_mask_files[0])
print(f"\n📸 Testando: {without_mask_files[0]}")
prever_mascara(img_sem_mascara, modelo)

## 💾 14. Salvando o Modelo

Agora vamos salvar o modelo em diferentes formatos:

1. **Keras (.keras)**: Formato nativo, fácil de carregar no Python
2. **TensorFlow Lite (.tflite)**: Formato otimizado para dispositivos embarcados como ESP32

In [ ]:
# Salva o modelo no formato Keras
modelo.save('modelo_mascara.keras')
print("✅ Modelo salvo como 'modelo_mascara.keras'")

# Também salva os pesos separadamente
modelo.save_weights('pesos_mascara.weights.h5')
print("✅ Pesos salvos como 'pesos_mascara.weights.h5'")

## 📱 15. Conversão para TensorFlow Lite

Para rodar na ESP32-CAM, precisamos converter o modelo para **TensorFlow Lite** e **quantizar** (converter de float32 para int8).

### Benefícios da Quantização:
- Reduz o tamanho do modelo em **~4x**
- Acelera a inferência na ESP32
- Usa menos memória RAM

In [ ]:
# Converte para TensorFlow Lite (sem quantização)
converter = tf.lite.TFLiteConverter.from_keras_model(modelo)
tflite_model = converter.convert()

# Salva o modelo TFLite
with open('modelo_mascara.tflite', 'wb') as f:
    f.write(tflite_model)

tamanho_tflite = len(tflite_model) / 1024
print(f"\n✅ Modelo TFLite salvo!")
print(f"   📦 Tamanho: {tamanho_tflite:.2f} KB")

In [ ]:
# Conversão com quantização INT8 (otimizado para ESP32)
def representative_dataset():
    """Gera dados representativos para calibração da quantização."""
    for i in range(min(100, len(X_train))):
        yield [X_train[i:i+1].astype(np.float32)]

# Configura o conversor para quantização completa
converter_quant = tf.lite.TFLiteConverter.from_keras_model(modelo)
converter_quant.optimizations = [tf.lite.Optimize.DEFAULT]
converter_quant.representative_dataset = representative_dataset
converter_quant.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_quant.inference_input_type = tf.uint8
converter_quant.inference_output_type = tf.uint8

try:
    tflite_model_quant = converter_quant.convert()
    
    # Salva o modelo quantizado
    with open('modelo_mascara_quant.tflite', 'wb') as f:
        f.write(tflite_model_quant)
    
    tamanho_quant = len(tflite_model_quant) / 1024
    print(f"\n✅ Modelo TFLite Quantizado (INT8) salvo!")
    print(f"   📦 Tamanho: {tamanho_quant:.2f} KB")
    print(f"   📉 Redução: {(1 - tamanho_quant/tamanho_tflite)*100:.1f}%")
except Exception as e:
    print(f"\n⚠️ Quantização INT8 completa falhou: {e}")
    print("   Usando quantização híbrida...")
    
    converter_hybrid = tf.lite.TFLiteConverter.from_keras_model(modelo)
    converter_hybrid.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model_hybrid = converter_hybrid.convert()
    
    with open('modelo_mascara_quant.tflite', 'wb') as f:
        f.write(tflite_model_hybrid)
    
    tamanho_hybrid = len(tflite_model_hybrid) / 1024
    print(f"\n✅ Modelo TFLite com quantização híbrida salvo!")
    print(f"   📦 Tamanho: {tamanho_hybrid:.2f} KB")

## 📊 16. Resumo Final

Vamos ver um resumo de tudo que foi feito!

In [ ]:
# Resumo final
print("\n" + "="*70)
print("📊 RESUMO DO PROJETO")
print("="*70)

print(f"""
🗂️ DATASET:
   • Total de imagens: {total_images}
   • Com máscara: {n_with_mask}
   • Sem máscara: {n_without_mask}

🖼️ PRÉ-PROCESSAMENTO:
   • Tamanho: {IMG_SIZE}x{IMG_SIZE} pixels
   • Canais: {'Grayscale' if IMG_CHANNELS == 1 else 'RGB'}
   • Normalização: 0-1

🧠 MODELO CNN:
   • Camadas convolucionais: 3
   • Filtros: 8 → 16 → 32
   • Total de parâmetros: {modelo.count_params():,}

📈 RESULTADOS:
   • Acurácia no teste: {test_accuracy*100:.2f}%
   • Loss no teste: {test_loss:.4f}

💾 ARQUIVOS GERADOS:
   • modelo_mascara.keras (Keras)
   • modelo_mascara.tflite (TensorFlow Lite)
   • modelo_mascara_quant.tflite (Quantizado para ESP32)
""")

print("="*70)
print("🎉 PROJETO CONCLUÍDO COM SUCESSO!")
print("="*70)

---

## 🚀 Próximos Passos (para a ESP32-CAM)

Quando você tiver a ESP32-CAM, os próximos passos serão:

1. **Converter o modelo para C array** (header file .h)
2. **Configurar o TensorFlow Lite Micro** na ESP32
3. **Capturar imagens da câmera** e pré-processar
4. **Rodar inferência** e mostrar resultado

### Comando para converter TFLite para C array:
```bash
xxd -i modelo_mascara_quant.tflite > modelo_mascara.h
```

---

**Autor:** Projeto de PDI - Detecção de Máscara Facial  
**Data:** Dezembro 2024